# What is the Z3 Solver 

In [1]:
# IMPORTS

In [4]:
import z3
import pkgutil
import os

In [15]:
for importer, modname, ispkg in pkgutil.iter_modules(z3.__path__):
    print(modname)

z3
z3consts
z3core
z3num
z3poly
z3printer
z3rcf
z3types
z3util


In [1]:
from z3.z3 import *

In [44]:
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Or(Tie, Shirt), Or(Not(Tie), Shirt), Or(Not(Tie), Not(Shirt)))
s.check() # sat = satisfying assignment for the formulas
s.model()

[Tie = False, Shirt = True]

In [6]:
"""
solve system of equations
3x+2y-z=1
2x-2y+4z=-2
-x+1/2y-z=0
"""
x = Real('x')
y = Real('y')
z = Real('z')
s = Solver()
s.add(3*x+2*y-z==1, 2*x-2*y+4*z==-2, -x+0.5*y-z==0)
while s.check() == sat:
    print(s.model())
    s.add(Or(x != s.model()[x], y != s.model()[y], z != s.model()[z]))

[y = -2, x = 1, z = -2]


In [59]:
circle, square, triangle = Ints('circle square triangle')
s = Solver()
s.add(circle+circle==10)
s.add(circle*square+square==12)
s.add(circle*square-triangle*circle==circle)
print(s.check())
print(s.model())

sat
[circle = 5, square = 2, triangle = 1]


In [ ]:
"""
linear diophantine equations
2 or more integer unknowns, each integer unknown is of degree 1
ax+by=d, 
this doesn't work, you need mk85
"""
s = Solver
a, b, c, d, e, f = BitVec('a', 16), BitVec('b', 16), BitVec('c', 16), BitVec('d', 16), BitVec('e', 16), BitVec('f', 16)
s.add(a<=10, b<=10, c<=10, d<=10, e<=10, f<=10)
s.add(a*215+b*275+c*335+d*355+e*420+f*580==1505)
while s.check():
    m = s.model()
    print(m)
    s.add(Not(And(a==m["a"], b==m["b"], c==m["c"], d==m["d"], e==m["e"], f==m["f"])))

AttributeError: 'BoolRef' object has no attribute 'assert_exprs'

In [72]:
"""
The sum of two nonzero real numbers is 4 times their product.
What is the sum of the reciprocals of the two numbers?
"""
x, y = Reals('x y')
s = Solver()
s.add(x>0)
s.add(y>0)
s.add(x+y==4*x*y)
s.check()
m = s.model()
print(m)
print('answer', m.evaluate(1/x+1/y))

[y = 1/3, x = 1]
answer 4


In [6]:
%%time
"""
Solving equation in advent of code
These are the first constraints
0<=x<=4000000
0<=y<=4000000
The additional constraints are the following

"""

def z3abs(x: int) -> int:
    return If(x>=0, x, -x)
manhattan = lambda x1, y1, x2, y2: abs(x1 - x2) + abs(y1 - y2)
sensors = [(13820, 3995710), (3286002, 2959504), (3654160, 2649422), (3702414, 2602790), (375280, 2377181), (3875726, 2708666), (3786107, 2547075), (2334266, 3754737), (1613400, 1057722), (3305964, 2380628), (1744420, 3927424), (3696849, 2604845), (2357787, 401688), (2127900, 1984887), (3705551, 2604421), (1783014, 2978242), (2536648, 2910642), (3999189, 2989409), (3939169, 2382534), (2792378, 2002602), (3520934, 3617637), (2614525, 1628105), (2828931, 3996545), (2184699, 2161391), (2272873, 1816621), (1630899, 3675405), (3683190, 2619409), (180960, 185390), (1528472, 3321640), (3993470, 2905566), (1684313, 20931), (2547761, 2464195), (3711518, 845968), (3925049, 2897039), (1590740, 3586256), (1033496, 3762565)]
beacons = [(1532002, 3577287), (3931431, 2926694), (3702627, 2598480), (3702627, 2598480), (2120140, 2591883), (3931431, 2926694), (3702627, 2598480), (2707879, 3424224), (1686376, -104303), (3702627, 2598480), (1532002, 3577287), (3702627, 2598480), (1686376, -104303), (2332340, 2000000), (3702627, 2598480), (2120140, 2591883), (2707879, 3424224), (3931431, 2926694), (3702627, 2598480), (2332340, 2000000), (2707879, 3424224), (2332340, 2000000), (2707879, 3424224), (2332340, 2000000), (2332340, 2000000), (1532002, 3577287), (3702627, 2598480), (187063, -1440697), (1532002, 3577287), (3931431, 2926694), (1686376, -104303), (2120140, 2591883), (3702627, 2598480), (3931431, 2926694), (1532002, 3577287), (1532002, 3577287)]
s = Solver()
x, y = Ints('x y')
s.add(x>=0, x<=4000000, y>=0, y<=4000000)
for sensor, beacon in zip(sensors, beacons):
    xs, ys = sensor
    xb, yb = beacon
    dist = manhattan(xs, ys, xb, yb)
    s.add(z3abs(x-xs) + z3abs(y-ys) > dist)
if s.check():
    model = s.model()
    print(model)

[x = 2939043, y = 2628223]
CPU times: user 214 ms, sys: 49.8 ms, total: 264 ms
Wall time: 845 ms


In [2]:

"""
constraint problem with bfs

Blueprint 1: Each ore robot costsgeore 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
"""
from parse import compile

pat = compile("Blueprint {:d}: Each ore robot costs {:d} ore. Each clay robot costs {:d} ore. Each obsidian robot costs {:d} ore and {:d} clay. Each geode robot costs {:d} ore and {:d} obsidian.")
class Blueprint:
    def __init__(self, id, oreore, clayore, obore, obclay, geore, geob):
        self.id = id
        self.oreore = oreore
        self.clayore = clayore
        self.obore = obore
        self.obclay = obclay
        self.geore = geore
        self.geob = geob
        
    def __repr__(self):
        return f'blueprint id: {self.id}, oreore: {self.oreore}, clayore: {self.clayore}, obore: {self.obore}, obclay: {self.obclay}, geore: {self.geore}, geob: {self.geob}'

s = "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian."
blueprint = Blueprint(*pat.parse(s))

In [11]:
from collections import deque
import sys
# sys.stdout = open('output.txt', 'w')
initial_state = (0, 0, 0, 0, 0, 0, 0, 1, 0) # geods count, geod robots count, obsidian robots count, clay robots count, 
# initial_state = (0, 8, 13, 3, 0, 2, 4, 1, 18)
queue = deque([initial_state]) # geods count, obisidan count, clay count, ore count, geod robots count, obsidian robots count, clay robots count, ore robots count, time 
best = 0
prev_time = 0
needed_ore = max(blueprint.oreore, blueprint.clayore, blueprint.obore, blueprint.geore)
needed_clay = blueprint.obclay
needed_obsidian = blueprint.geob
sol = []
cnt = 2
while queue:
    geods, obsidian, clay, ore, geod_robots, obsidian_robots, clay_robots, ore_robots, time = queue.popleft()
    state = (geods, obsidian, clay, ore, geod_robots, obsidian_robots, clay_robots, ore_robots, time)
    if (0, 3, 17, 2, 1, 2, 4, 1, 18) == state:
        sol.append(state)
    if time == 19: break
    if time > prev_time:
        print(time, len(queue), best)
        prev_time = time
    if time == 24:
        best = max(best, geods)
        continue
    if ore >= cnt*needed_ore or ore_robots >= needed_ore or clay_robots >= needed_clay or obsidian_robots >= needed_obsidian or clay > 8*needed_clay: 
        continue 
    s = Solver()
    obsidian_count, clay_count, ore_count, geod_robot_count, obsidian_robot_count, clay_robot_count, ore_robot_count = Ints('obsidian_count clay_count ore_count geod_robot_count obsidian_robot_count clay_robot_count ore_robot_count')
    s.add(obsidian_count <= obsidian, clay_count <= clay, ore_count <= ore)
    s.add(ore_robot_count >= 0, clay_robot_count >= 0, obsidian_robot_count >= 0, geod_robot_count >= 0)
    s.add(blueprint.oreore*ore_robot_count + blueprint.clayore*clay_robot_count + blueprint.obore*obsidian_robot_count + blueprint.geore*geod_robot_count == ore_count)
    s.add(blueprint.obclay*obsidian_robot_count == clay_count)
    s.add(blueprint.geob*geod_robot_count == obsidian_count)
    s.add(ore_robot_count + clay_robot_count + obsidian_robot_count + geod_robot_count <= 1)
    geod_robot_req = If(And(obsidian >= blueprint.geob, ore >= blueprint.geore), geod_robot_count == 1, geod_robot_count == 0)
    s.add(geod_robot_req)
    obsidian_robot_req = If(And(Or(obsidian < blueprint.geob, ore < blueprint.geore), clay >= blueprint.obclay, ore >= blueprint.obore), obsidian_robot_count == 1, obsidian_robot_count == 0)
    s.add(obsidian_robot_req)

    
    while s.check() == sat:
        next_geods, next_obsidian, next_clay, next_ore, next_geod_robots, next_obsidian_robots, next_clay_robots, next_ore_robots, next_time = geods + geod_robots, obsidian + obsidian_robots - s.model()[obsidian_count].as_long(), clay + \
        clay_robots - s.model()[clay_count].as_long(), ore + ore_robots - s.model()[ore_count].as_long(), geod_robots + s.model()[geod_robot_count].as_long(), obsidian_robots + s.model()[obsidian_robot_count].as_long(), clay_robots + s.model()[clay_robot_count].as_long(), ore_robots + \
        s.model()[ore_robot_count].as_long(), time + 1
        s.add(Or(obsidian_count != s.model()[obsidian_count], clay_count != s.model()[clay_count], ore_count != s.model()[ore_count], geod_robot_count != s.model()[geod_robot_count], obsidian_robot_count != s.model()[obsidian_robot_count], clay_robot_count != s.model()[clay_robot_count], ore_robot_count != s.model()[ore_robot_count]))
        next_state = (next_geods, next_obsidian, next_clay, next_ore, next_geod_robots, next_obsidian_robots, next_clay_robots, next_ore_robots, next_time)
        queue.append(next_state)


1 0 0
2 0 0
3 1 0
4 2 0
5 6 0
6 11 0
7 28 0
8 56 0
9 139 0
10 304 0
11 725 0
12 1474 0
13 2822 0
14 4345 0
15 6056 0
16 11070 0


KeyboardInterrupt: 

In [ ]:
sol

In [8]:
8*4*14*7

250880

In [13]:
# how to use the z3 solver and statement and if
geod_robot_count = Ints('geod_robot_count')
obsidian = blueprint.geob
ore = blueprint.geore 
geod_robot_req = If(And(obsidian >= blueprint.geob, ore >= blueprint.geore), geod_robot_count == 1, geod_robot_count == 0)
print(geod_robot_req)

If(And(True, True), False, False)


In [14]:
dir(geod_robot_req)

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'num_args',
 'params',
 'serialize',
 'sexpr',
 'sort',
 'sort_kind',
 'translate',
 'use_pp']

In [15]:
simplify(geod_robot_req)

False

In [ ]:
sys.stdout.close()

In [22]:
24**10

63403380965376

In [112]:
x, y = Ints('x y')
# x.eq = 2
# y.eq = 3
req = If(x==2, y > 0, y > 0)
req2 = If(And(Not(req), y == 3), y > 0, y > 0)


In [113]:
ss = Solver()

In [114]:
ss.add(x == 3, y == 3)
ss.add(req)
ss.add(req2)

In [115]:
ss.check()
ss.model()

[y = 3, x = 3]